<a href="https://colab.research.google.com/github/MartonJToth/KepszintezisLabor/blob/master/KepszintezisLabor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install ellenőrzés

In [1]:
!pip3 install torch torchvision 
!pip3 install Pillow==4.2.1

In [2]:
!nvcc --version
import torch
print(torch.cuda.is_available())

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
True


Adatbázis letöltés

In [3]:
!wget http://cg.iit.bme.hu/~tmarton/deeplearning/DehazeImages.zip
!unzip -qq DehazeImages.zip
!rm DehazeImages.zip

--2019-04-20 18:04:11--  http://cg.iit.bme.hu/~tmarton/deeplearning/DehazeImages.zip
Resolving cg.iit.bme.hu (cg.iit.bme.hu)... 152.66.241.108
Connecting to cg.iit.bme.hu (cg.iit.bme.hu)|152.66.241.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 795080607 (758M) [application/zip]
Saving to: ‘DehazeImages.zip’

DehazeImages.zip    100%[===================>] 758.25M  12.9MB/s    in 1m 49s  

2019-04-20 18:06:01 (6.95 MB/s) - ‘DehazeImages.zip’ saved [795080607/795080607]



DataLoader

In [0]:
import os
import imageio
import numpy as np
import torch
import random
from torch.utils.data import Dataset
import PIL
from PIL import Image, ImageCms


class DehazeDataset(Dataset):

    train_input = []
    train_results = []

    validation_input = []
    validation_results = []

    trainNum = 0
    valNum = 0
    width = 0
    height = 0

    def __init__(self, path, type='training', percentage='0.8', transform=None):

        self.sampleNum = 0
        self.width = 0
        self.height = 0
        self.channels = 3
        self.type = type

        self.transform = transform

        self.srgb_profile = ImageCms.createProfile("sRGB")
        self.lab_profile = ImageCms.createProfile("LAB")

        self.rgb2lab_transform = ImageCms.buildTransformFromOpenProfiles(self.srgb_profile, self.lab_profile, "RGB", "LAB")

        if len(DehazeDataset.train_input) == 0:

            numFiles = len([name for name in os.listdir(path) if os.path.isfile(os.path.join(path, name))])

            first = True

            for i in range(numFiles // 2):

                istr = str(i)

                while len(istr) < 5:
                    istr = '0' + istr

                inputPath = os.path.join(path, 'hazy_' + istr + '.png')
                resultPath = os.path.join(path, 'normal_' + istr + '.png')

                if first:
                    first = False
                    im = imageio.imread(uri=inputPath)
                    DehazeDataset.width = im.shape[0]
                    DehazeDataset.height = im.shape[1]

                if random.random() <= percentage:

                    DehazeDataset.train_input.append(inputPath)
                    DehazeDataset.train_results.append(resultPath)

                else:

                    DehazeDataset.validation_input.append(inputPath)
                    DehazeDataset.validation_results.append(resultPath)

        self.width = DehazeDataset.width // 4
        self.height = DehazeDataset.height // 4

        if type == 'training':
            self.sampleNum = len(DehazeDataset.train_input)
        else:
            self.sampleNum = len(DehazeDataset.validation_input)

    def __len__(self):
        return self.sampleNum

    def __getitem__(self, idx):

        x = np.zeros((self.width, self.height, self.channels), dtype='f')
        y = np.zeros((self.width, self.height, self.channels), dtype='f')

        if self.type == 'training':

            inputImage = Image.open(DehazeDataset.train_input[idx])
            resultImage = Image.open(DehazeDataset.train_results[idx])

        else:

            inputImage = Image.open(DehazeDataset.validation_input[idx])
            resultImage = Image.open(DehazeDataset.validation_results[idx])

        inputImage = inputImage.resize((self.width, self.height), PIL.Image.ANTIALIAS)
        inputImage = inputImage.convert('RGB')
        resultImage = resultImage.resize((self.width, self.height), PIL.Image.ANTIALIAS)
        resultImage = resultImage.convert('RGB')

        inputImage = np.array(inputImage)
        inputImage = np.true_divide(inputImage, 255)
        resultImage = np.array(resultImage)
        resultImage = np.true_divide(resultImage, 255)

        x[:, :, :] = inputImage
        y[:, :, :] = resultImage

        x = np.transpose(x, (2, 1, 0))
        y = np.transpose(y, (2, 1, 0))

        sample = {'input': x, 'output': y}

        if self.transform:
            sample = self.transform(sample)

        return sample
      
      


Transform

In [0]:
class ToTensor(object):

    def __call__(self, sample):
        input, output = sample['input'], sample['output']

        return {'input': torch.from_numpy(input),
                'output': torch.from_numpy(output)}

